# Notebook responsável por alguns aspectos de modelagem OneToOne / OneToFew Embedding (usando MongoEngine)

## Bibliotecas importantes

In [1]:
from mongoengine import *
import pprint
import numpy as np
import pandas as pd
import unidecode

# para geração de dados fake
from faker import Faker


## Classes úteis

In [2]:
class Endereco (EmbeddedDocument):
    logradouro = StringField(max_length=200, required=True)
    numero = StringField(max_length=10, required=True)
    bairro = StringField(max_length=50, required=True)
    cidade = StringField(max_length=200, required=True)
    uf = StringField(max_length=50, required=True)
    cep = StringField(max_length=10, required=True)
    
    def __str__(self):
        return f"{self.logradouro}, {self.numero}"
    
    
class Pessoa (Document):
    cpf = StringField(max_length=20, required=True)
    primeiro_nome = StringField(max_length=200, required=True)
    ultimo_nome = StringField(max_length=400, required=True)
    idade = IntField(min_value=0, max_value=120, required=True)
    data_nascimento = StringField(max_length=20, required=True)
    email = EmailField(max_length=50)
    telefones = ListField(StringField(max_length=50), max_length=4)
    formacoes = ListField(StringField(max_length=50), max_length=4)
    
    # Referência (linking) para um objeto da classe pessoa
    endereco = EmbeddedDocumentField(Endereco)
    
    def __str__(self):
        return f"{self.pk}, {self.cpf}, {self.primeiro_nome}, Endereço: {self.endereco}"


## Funções uteis

In [3]:
def criar_endereco():
    
    logradouro = fake.street_name()
    numero = fake.building_number()
    bairro = fake.bairro()
    cidade = fake.city()
    uf = fake.administrative_unit()
    cep = fake.postcode()

    endereco = Endereco(logradouro=logradouro, numero=numero, bairro=bairro, cidade=cidade, uf=uf, cep=cep)
    return endereco

In [4]:
def criar_pessoa():
    
    cpf = fake.cpf()
    primeiro_nome = fake.first_name()
    ultimo_nome = fake.last_name()
    idade = np.random.randint(18, 50)
    data_nascimento = fake.date()
    email = f'{cpf}@{fake.free_email_domain()}'
    
    # criando os telefones
    telefones = []
    n = np.random.randint(1, 4)
    for i in range(n):
        telefones.append(fake.msisdn())
    
    # criando as formações
    formacoes = []
    n = np.random.randint(1, 4)
    for i in range(n):
        formacoes.append(fake.job())
    
    endereco = criar_endereco()
    
    pessoa = Pessoa(cpf=cpf, primeiro_nome=primeiro_nome, 
                    ultimo_nome=ultimo_nome, idade=idade, 
                    data_nascimento=data_nascimento, 
                    email=email, telefones=telefones, 
                    formacoes=formacoes, endereco=endereco)
    return pessoa

## Configurações

In [5]:
pp = pprint.PrettyPrinter(compact=True)
# gerando dados fictícios em português do Brasil
fake = Faker(['pt_BR'])

## Inserção de vários documentos

### Conectando ao MongoDB

In [7]:
connect('onetoone_embedding')

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### Exemplo - Criação de vários documentos compostos

In [9]:
lista_pessoas = []

for i in range(10):
    lista_pessoas.append(criar_pessoa())
    
pp.pprint(lista_pessoas)

[<Pessoa: None, 901.456.327-25, Júlia, Endereço: Avenida de Gonçalves, 86>,
 <Pessoa: None, 058.719.423-50, Sofia, Endereço: Praça Davi Luiz Viana, 80>,
 <Pessoa: None, 401.829.537-50, Danilo, Endereço: Esplanada de Ferreira, 22>,
 <Pessoa: None, 902.164.738-96, Laura, Endereço: Chácara de Jesus, 51>,
 <Pessoa: None, 859.671.230-59, Pedro Lucas, Endereço: Loteamento Noah da Rosa, 51>,
 <Pessoa: None, 504.763.891-00, Amanda, Endereço: Jardim Ana Luiza Vieira, 37>,
 <Pessoa: None, 207.913.568-68, Ana Luiza, Endereço: Ladeira Heitor da Cruz, 72>,
 <Pessoa: None, 084.152.639-70, André, Endereço: Campo João Vitor Moraes, 39>,
 <Pessoa: None, 829.465.713-19, Vinicius, Endereço: Chácara Pinto, 8>,
 <Pessoa: None, 140.952.763-80, Rafael, Endereço: Conjunto de Rocha, 37>]


In [10]:
pp.pprint(lista_pessoas[0].pk)

None


### Inserindo os documentos no banco de dados

In [11]:
# inserindo 1 documento apenas
lista_pessoas[0].save()

<Pessoa: 625ff64041ea728155610666, 901.456.327-25, Júlia, Endereço: Avenida de Gonçalves, 86>

In [12]:
pp.pprint(lista_pessoas[0].pk)

ObjectId('625ff64041ea728155610666')


In [13]:
# inserindo todos os outros documentos
Pessoa.objects.insert(lista_pessoas[1:])

[<Pessoa: 625ff67341ea728155610667, 058.719.423-50, Sofia, Endereço: Praça Davi Luiz Viana, 80>,
 <Pessoa: 625ff67341ea728155610668, 401.829.537-50, Danilo, Endereço: Esplanada de Ferreira, 22>,
 <Pessoa: 625ff67341ea728155610669, 902.164.738-96, Laura, Endereço: Chácara de Jesus, 51>,
 <Pessoa: 625ff67341ea72815561066a, 859.671.230-59, Pedro Lucas, Endereço: Loteamento Noah da Rosa, 51>,
 <Pessoa: 625ff67341ea72815561066b, 504.763.891-00, Amanda, Endereço: Jardim Ana Luiza Vieira, 37>,
 <Pessoa: 625ff67341ea72815561066c, 207.913.568-68, Ana Luiza, Endereço: Ladeira Heitor da Cruz, 72>,
 <Pessoa: 625ff67341ea72815561066d, 084.152.639-70, André, Endereço: Campo João Vitor Moraes, 39>,
 <Pessoa: 625ff67341ea72815561066e, 829.465.713-19, Vinicius, Endereço: Chácara Pinto, 8>,
 <Pessoa: 625ff67341ea72815561066f, 140.952.763-80, Rafael, Endereço: Conjunto de Rocha, 37>]

## Recuperando os documentos inseridos

#### Recuperar todos

In [14]:
for p in Pessoa.objects:
    pp.pprint(p.id)

ObjectId('625ff64041ea728155610666')
ObjectId('625ff67341ea728155610667')
ObjectId('625ff67341ea728155610668')
ObjectId('625ff67341ea728155610669')
ObjectId('625ff67341ea72815561066a')
ObjectId('625ff67341ea72815561066b')
ObjectId('625ff67341ea72815561066c')
ObjectId('625ff67341ea72815561066d')
ObjectId('625ff67341ea72815561066e')
ObjectId('625ff67341ea72815561066f')


In [15]:
pessoa = Pessoa.objects.first()
pp.pprint(pessoa)

<Pessoa: 625ff64041ea728155610666, 901.456.327-25, Júlia, Endereço: Avenida de Gonçalves, 86>


In [16]:
pessoa.delete()

In [17]:
Pessoa.objects(id='625f455401e31f0f776785f0')

[]